In [2]:
import serial
import struct
import time

def envio_serial(valor_float):
    # Codifica el float en 4 bytes
    #data_bytes = struct.pack('f', valor_float)
    #ser.write(data_bytes)
    ser.write(b'F' + struct.pack('f', valor_float))

def recep_serial(timeout=2.0, tipo='float'):
    start_time = time.time()
    
    while True:
        if ser.in_waiting >= 4 if tipo == 'float' else ser.in_waiting > 0:
            break
        if time.time() - start_time > timeout:
            print("Tiempo de espera agotado")
            return None

    if tipo == 'float':
        data = ser.read(4)
        if len(data) < 4:
            print("Advertencia: no se recibieron 4 bytes")
            return None
        return struct.unpack('f', data)[0]
    
    elif tipo == 'texto':
        linea = ser.readline().decode('utf-8', errors='ignore').strip()
        return linea

    else:
        print("Tipo de dato no reconocido")
        return None

In [2]:
# Configuración del puerto serie
ser = serial.Serial(
    #port='/dev/ttyUSB0',              # Ajustar al puerto correcto
    port="COM5",
    baudrate=115200
    #bytesize=serial.EIGHTBITS,
    #parity=serial.PARITY_NONE,
    #stopbits=serial.STOPBITS_ONE,
    #timeout=1
)

In [4]:

def envio_serial(valor_float):
    # Codifica el float en 4 bytes
    #data_bytes = struct.pack('f', valor_float)
    #ser.write(data_bytes)
    ser.write(b'F' + struct.pack('f', valor_float))

def recep_serial():
    # Lee 4 bytes del puerto serial y decodifica a float
    data = ser.read(4)
    if len(data) < 4:
        print("Advertencia: no se recibieron 4 bytes")
        return 0.0
    valor_float = struct.unpack('f', data)[0]
    return valor_float

In [4]:
envio_serial(0)
recep_serial()

5.5674765078335015e-39

In [7]:
import serial
import struct
import time

def envio_serial(ser,valor_float):
    # Codifica el float en 4 bytes
    #data_bytes = struct.pack('f', valor_float)
    #ser.write(data_bytes)
    ser.write(b'F' + struct.pack('f', valor_float))


def leer_serial_2segundos(puerto, baud_rate=115200, duracion=2, enviar=0):
    # Configurar el puerto serial
    ser = serial.Serial(puerto, baud_rate, timeout=1)
    envio_serial(ser,enviar)
    # Guardar el tiempo de inicio
    start_time = time.time()
    
    # Almacenar todo el texto recibido
    texto_recibido = ""

    # Leer hasta que pasen 2 segundos
    while time.time() - start_time < duracion:
        if ser.in_waiting > 0:  # Si hay datos disponibles
            texto_recibido += ser.read(ser.in_waiting).decode('utf-8', errors='ignore')  # Leer y decodificar
    
    # Cerrar el puerto serial
    ser.close()

    return texto_recibido

In [22]:
texto = leer_serial_2segundos('COM5',115200,0.5,65)  # Ajusta el puerto 'COM3' a tu configuración
print("Texto recibido durante 2 segundos:")
print(texto)

Texto recibido durante 2 segundos:
error: 65.00, e_old1: 65.00, e_old2: 65.00, control_old: 108.24
control: 108.24
yB


In [ ]:
595.65
108.24

In [25]:
# Buffer de salidas
Y1 = 25
Y2 = 25
# Buffer de entradas
X1 = 108.24
X2 = 595.65


pA = -0.04056   # 2.373e-6
pB =  0.04962   # 2.357e-6
pC = 1.81
pD = -0.8187


salida = pA*X1 + pB*X2 + pC*Y1 + pD * Y2
salida

49.9484386

In [ ]:
import serial
import struct
import time

def recep_serial(puerto, baud_rate=9600, timeout=0.2):
    """
    Lee 4 bytes del puerto serial con delimitadores 'R' y 'C'.
    Si no se detectan en 0.2 segundos, limpia el buffer y reintenta.

    Parámetros:
        - puerto: El puerto serial, como 'COM3' o '/dev/ttyUSB0'.
        - baud_rate: Velocidad de transmisión serial.
        - timeout: Tiempo en segundos para esperar los delimitadores.

    Retorna:
        - valor_float: El valor float recibido o None si no se reciben datos válidos.
    """
    # Abrir puerto serial
    ser = serial.Serial(puerto, baud_rate, timeout=timeout)
    
    # Tiempo de inicio
    start_time = time.time()

    # Buffer de entrada
    buffer = ""

    while True:
        if ser.in_waiting > 0:  # Si hay datos en el puerto serial
            char = ser.read(1).decode('utf-8', errors='ignore')

            # Agregar al buffer
            buffer += char

            # Si encontramos 'R', comenzamos a esperar los datos
            if buffer == "R":
                # Esperamos los 4 bytes del float
                data = ser.read(4)

                # Si no recibimos 4 bytes, vaciamos el buffer y reintentamos
                if len(data) < 4:
                    print("Advertencia: no se recibieron 4 bytes, limpiando buffer.")
                    ser.reset_input_buffer()  # Vaciar el buffer de entrada
                    buffer = ""  # Reiniciar el buffer
                    continue  # Volver a empezar el ciclo

                # Si recibimos 4 bytes, los convertimos a float
                valor_float = struct.unpack('f', data)[0]

                # Esperamos el delimitador 'C'
                char = ser.read(1).decode('utf-8', errors='ignore')
                if char == "C":
                    # Si encontramos 'C', hemos recibido el mensaje completo
                    return valor_float
                else:
                    # Si no encontramos 'C', vaciamos el buffer y reintentamos
                    print("Advertencia: No se recibió el delimitador 'C', limpiando buffer.")
                    ser.reset_input_buffer()  # Vaciar el buffer de entrada
                    buffer = ""  # Reiniciar el buffer

        # Si pasaron más de 0.2 segundos sin recibir los delimitadores, vaciar el buffer
        if time.time() - start_time > timeout:
            print("Tiempo de espera agotado, limpiando buffer.")
            ser.reset_input_buffer()  # Vaciar el buffer de entrada
            buffer = ""  # Reiniciar el buffer
            start_time = time.time()  # Reiniciar el contador de tiempo

# Ejemplo de uso
puerto = 'COM3'  # Ajusta según el puerto de tu dispositivo
valor = recep_serial(puerto)
print(f"Valor recibido: {valor}")
